In [ ]:
# Libraries
import numpy as np
import cv2
import pandas as pd
import os

from keras import layers
from keras.preprocessing import image
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import  MaxPooling2D, Dropout

from keras.models import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image
import efficientnet.keras as efn

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras import optimizers

In [ ]:
# We'll change the name of image id with image extensions
df = pd.read_csv('train.csv')
df.head()

In [ ]:
# small function to append extension 
def append_ext(fn):
    return fn+".jpg"

In [ ]:
df["image_id"]=df["image_id"].apply(append_ext)

In [ ]:
# This is used for data augmentation, data flow without taking up too much memory,resizing the images
# as well and splitting into training and validation sets and coverting labels to categorical values
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        featurewise_center= False,
        featurewise_std_normalization=False,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        zca_epsilon=1e-06,
        brightness_range = (0.1,0.3),
        validation_split = 0.20) # comment this line in case of inceptionv3 

In [ ]:
# Augmentor for train.csv , flow from dataframe will read each image name from dataframe and take 
# the corresponding image directly from directory
train_generator=datagen.flow_from_dataframe(
        dataframe=df,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224)) # change the lines as per input dimension of the model

In [ ]:
# Do not run this in case of inception v3
valid_generator=datagen.flow_from_dataframe(
        dataframe=df,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical",
        target_size=(224,224)) # change this line as per dimensions of each model

In [ ]:
base_model =   applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#base_model =   applications.InceptionV3(weights='imagenet', include_top= False)
#base_model =   applications.VGG16(weights='imagenet', include_top= True)
#base_model =   applications.InceptionResNetV2(weights='imagenet', include_top= True)
#base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [ ]:
# In cases where we do not use the top layer
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.3))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(35, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

#model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#              metrics=['accuracy'])

In [ ]:
# For efficientNet
for layer in base_model.layers:
    layer.trainable = False
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(35, activation="sigmoid")(x)
model_final = Model(inputs = model.input, outputs = predictions)
model_final.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])
eff_history = model_final.fit_generator(train_generator, steps_per_epoch = 81, epochs = 25)

In [ ]:
base_model.summary()

In [ ]:
# Use this for resnet and its derivatives
x=Base_Model.layers[-2].output
fc1=Dense(35,activation='softmax')(x)

In [ ]:
my_model=Model(inputs=base_model.input,outputs=fc1)

In [ ]:
# Adam works just as well, did not get good yeild from SGD in this case
from keras.optimizers import RMSprop
optimizer=RMSprop(lr=0.0001, decay=1e-6)

In [ ]:
optimizer = optimizers.SGD(lr=1e-4, momentum=0.9)

In [ ]:
# Make the layers untrainable 48 is for InceptionV3
# 164 for inception resnetV2
# 50 for resnet50
# 16 for vgg
# This makes sure that the layers do not get changed during training
for l in my_model.layers[:-48]:
    #print(l)
    l.trainable = False
    
my_model.compile(optimizer=optimizer,loss ="categorical_crossentropy",
                 metrics=["accuracy"])

In [ ]:
cd kaggle/input/hackerearths-snakes-in-the-hood/dataset

In [ ]:
my_model.fit_generator(train_generator,epochs=25,
                       #validation_data=valid_generator,
                       #validation_steps=20,
                       steps_per_epoch=5508//128)

In [ ]:
test_df=pd.read_csv('test.csv')

In [ ]:
test_df["image_id"]=test_df["image_id"].apply(append_ext)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,)

test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory="test/",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))

In [ ]:
# Predict the output
y_pred=my_model.predict_generator(test_generator,verbose=1)
y_pred_2=np.argmax(y_pred,axis=1)

In [ ]:
# Prepare for saving
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]

In [ ]:
test_df=pd.read_csv('test.csv')
test_name=test_df['image_id'].values
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

In [ ]:
# Use cd / before this is it doesnt work, we need to go to wokring directory of kaggle to save
cd kaggle/working

In [ ]:
data.to_csv('output3.csv',index=False)

It is observed that inceptionV3 and InceptionResnetV2 give the best outputs for 25 epochs,
Self created CNN did not give accuracy above 9% whatsoever hence transfer learning was saught after.
Methods and Models used here are referred from documentations and examples as available 
Further comments and suggestions are welcome.....questions if any are also welcome